
Questions to be considered for the project 

what industriess performed best over a certain time period?  
Are there trends in stock price movements by quarter, year, decade?  
Can we predict stock price trends using historical data?  
Impact of certain events on the stock prices i.e. US election, end of quarter performance announcements by companies, press releases etc

In [65]:
# importing libraries

import pandas as pd
import yfinance as yf
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt

In [66]:
# creating a dictionary to define the industries and the companies that will be used for the project
industries = {
    'Technology': ['AAPL', 'MSFT', 'NVDA'], 
    'Quantum Computing': ['IONQ', 'RGTI', 'QBTS'],
    'Electric Vehicles': ['TSLA', 'RIVN', 'LCID'],
    'Renewable Energy': ['NEE', 'FSLR', 'ENPH']
}

In [67]:
# creating a dictionary to store dataFrames to for different time period
industry_dataframes = {
    '5y': pd.DataFrame(),  # mid-term trends (5 years)
    '1y': pd.DataFrame(),  # quarterly/yearly movements (1 year)
    'event_specific': pd.DataFrame(),  # specific date ranges for event analysis 2024 US election
    '10y': pd.DataFrame()  # long-term trends (10 years or max)
}

## Checking the data

Due to the newness of some of the industries being considered for the project.  
The timeframe the data is available for each of the choosen companies will be checked to see if they have data available for the last 5 years. 

In [68]:
print(f'Company:\tData starts:\tLatest date:\t5yr available?')
print(f'------------------------------------------------------------------')

# checking the start date of the data for the chosen companies
for industry, companies in industries.items():
        for company in companies:
            ticker = yf.Ticker(company)
            hist = ticker.history(period='max')  # fetching the maximum available data
            start_date = hist.index.min().strftime('%Y-%m-%d')  # getting the earliest date available but looking at the min in the index for each company and formating the start date as a string to be shown in so only the date information and not the timestamp are included
            latest_date = hist.index.max().strftime('%Y-%m-%d')
            if pd.to_datetime(start_date) > pd.Timestamp('2019-12-27'):
                five_yr_available = 'No' 
            else: 
                five_yr_available = 'Yes'
            
            print(f"{company}\t\t{start_date}\t{latest_date}\t{five_yr_available}") # printing the company, date the data is available from, latest date information for the data pulled and a yes/no answer of if the data is available for the last 5 years


Company:	Data starts:	Latest date:	5yr available?
------------------------------------------------------------------
AAPL		1980-12-12	2024-12-27	Yes
MSFT		1986-03-13	2024-12-27	Yes
NVDA		1999-01-22	2024-12-27	Yes
IONQ		2021-01-04	2024-12-27	No
RGTI		2021-04-22	2024-12-27	No
QBTS		2020-12-11	2024-12-27	No
TSLA		2010-06-29	2024-12-27	Yes
RIVN		2021-11-10	2024-12-27	No
LCID		2020-09-18	2024-12-27	No
NEE		1973-02-21	2024-12-27	Yes
FSLR		2006-11-17	2024-12-27	Yes
ENPH		2012-03-30	2024-12-27	Yes


From the above we can see that there are several companies which do not have 5 years worth of data available.  
These are companies belowing to Quantum computing & electronic vechile industries, which is not surprising as the technology for these industries are relatively new/young in comparison to some of the historical companies/industries in the stock market.  

With this information the technology and renewable energy industry will be used for mid-long term analysis. 

In [69]:

# fetching and saving the data for multiple time periods to individual dataframes
for time_period in ['5y', '1y', 'event_specific', '10y']:
    temp_data = []  # storing the data temporarily for each time period so it can be added to the dataframe later
    for industry, companies in industries.items():  # fetching the data for each industry by looping through the key value pairs returned from the .items() method
        for company in companies:  # getting the data for each company within the industry specified for the parent for loop
            # fetching the historical data
            # if loop checking the time_period value per iteration and choosing what steps to complete depending on the value 
            if time_period == 'event_specific': # the code will follow this path if the time period matches event_specific
                hist = yf.Ticker(company).history(start='2023-12-26', end='2024-12-26') # getting the info for the date range specified 
            elif time_period in ['5y', '10y'] and industry in ['Quantum Computing', 'Electric Vehicles']: 
                continue  # skipping fetching 5y and 10y data for the Quantum Computing and Electric Vehicles industries as not all companies have sufficient data for this
            else:
                hist = yf.Ticker(company).history(period=time_period)
            # adding headers and defining which data should be entered for those columns
            hist['Industry'] = industry
            hist['Company'] = company
            hist['Timeframe'] = time_period
            temp_data.append(hist) # adding the details gathered to the temp_data list
    # adding all data for the period into a DataFrame
    industry_dataframes[time_period] = pd.concat(temp_data)
    
# print(industry_dataframes['1y'])
# checking which industries the data is present in the dataframes to ensure the if statement has executed successfully
# commented out for now as testing was successful
# counted_industries_1 = industry_dataframes['1y']['Industry'].value_counts()
# print(f'1 year count:\n{counted_industries_1}\n')
# counted_industries_5 = industry_dataframes['5y']['Industry'].value_counts()
# print(f'5 year count:\n{counted_industries_5}\n')
# counted_industries_es = industry_dataframes['event_specific']['Industry'].value_counts()
# print(f'Event specific count:\n{counted_industries_es}\n')
# counted_industries_10 = industry_dataframes['10y']['Industry'].value_counts()
# print(f'10 year count:\n{counted_industries_10}')


### Checking DataFrames for missing data and the data types of the columns


In [70]:
# lopping through each of the dataframes created to see if any data is missing
for time_period in ['5y', '1y', 'event_specific', '10y']: 
    no_of_blanks = industry_dataframes[time_period].isna().sum().sum() # looking for NaN values and getting the total of the sum of missing values for all columns in the df 
    print(f'The {time_period} data contains {no_of_blanks} missing values.') # printing one summary line to state if there is missing values in the dataset
    df_dtypes = industry_dataframes[time_period].dtypes.value_counts()
    print(f'The following data types can be found in the df:\n{df_dtypes}\n')

The 5y data contains 0 missing values.
The following data types can be found in the df:
float64    6
object     3
int64      1
Name: count, dtype: int64

The 1y data contains 0 missing values.
The following data types can be found in the df:
float64    6
object     3
int64      1
Name: count, dtype: int64

The event_specific data contains 0 missing values.
The following data types can be found in the df:
float64    6
object     3
int64      1
Name: count, dtype: int64

The 10y data contains 0 missing values.
The following data types can be found in the df:
float64    6
object     3
int64      1
Name: count, dtype: int64



From the above we can see there are 3 columns of data type ``object``.  
As calculations and comparitive analysis will be performed later in the project, next the 3 columns which are of dtype ``object`` will be checked to ensure these are not numeric columns which have incorrect dtype assigned. This is done as a value assigned as an ``object`` will not be possible to be used for performing numerical operations later. 

In [71]:
# checking which columns are of dtype object
print('The columns of dtype object are:\n')
for time_period in ['5y', '1y', 'event_specific', '10y']: 
    # Find columns with 'object' data type
    object_cols = industry_dataframes[time_period].select_dtypes(include='object').columns.tolist()
    print(f'{time_period.capitalize()} DataFrame:\n{object_cols}\n')


The columns of dtype object are:

5y DataFrame:
['Industry', 'Company', 'Timeframe']

1y DataFrame:
['Industry', 'Company', 'Timeframe']

Event_specific DataFrame:
['Industry', 'Company', 'Timeframe']

10y DataFrame:
['Industry', 'Company', 'Timeframe']



As the 'Industry', 'Company' and 'Timeframe' are not numeric columns and not needed for calculations, the ``object`` dtype should cause no issues.

### Checking the index data
As analysis and comparisons on the data over different time periods will be performed, having the index be of ``Pandas`` ``datetime`` index is preferred.

In [72]:
# checking the index dtype for each dataframe
for time_period in ['5y', '1y', 'event_specific', '10y']:
    index_dtype = industry_dataframes[time_period].index.dtype
    print(f'The index data type for the {time_period} dataframe is: {index_dtype}')

The index data type for the 5y dataframe is: datetime64[ns, America/New_York]
The index data type for the 1y dataframe is: datetime64[ns, America/New_York]
The index data type for the event_specific dataframe is: datetime64[ns, America/New_York]
The index data type for the 10y dataframe is: datetime64[ns, America/New_York]


From the above output it can be seen that the index data is of the datetime64 dtype which is desired. However, additional information is present suggesting the timezone of the data retrieved is America/New York and not UTC.  
To confirm this, one of the index values will be checked as an example to see how the data is showing.

In [73]:
# printing the first value for the index in each df
print(f'The first index value in each index is:')
for time_period in ['5y', '1y', 'event_specific', '10y']:
    print(f'{time_period}: {industry_dataframes[time_period].index[0]}')  # First index


The first index value in each index is:
5y: 2019-12-30 00:00:00-05:00
1y: 2023-12-28 00:00:00-05:00
event_specific: 2023-12-26 00:00:00-05:00
10y: 2014-12-29 00:00:00-05:00


The timestamp ending in ``-05:00`` confirms that the timestamp is matching the ``ns, America/New_York`` timezone.  
As UTC is typically the standard timezone used worldwide the index will be updated to convert from ``America/New_York`` to ``UTC`` to tidy the data.  
This will also be beneficial to avoid any issues with compatibility should any further markets be brought in scope of the project further down the line.

In [75]:

for time_period in ['5y', '1y', 'event_specific', '10y']:
    industry_dataframes[time_period].index = industry_dataframes[time_period].index.tz_convert('UTC')
    print(f'{time_period}: {industry_dataframes[time_period].index[0]}')  # First index



5y: 2019-12-30 05:00:00+00:00
1y: 2023-12-28 05:00:00+00:00
event_specific: 2023-12-26 05:00:00+00:00
10y: 2014-12-29 05:00:00+00:00


In [ ]:
# creating an empty list to store summary statistics for each industry
industry_summary = []

# fetching the data for each industry by looping through the key value pairs returned from the .items() method
for industry, companies in industries.items():
    data = yf.Tickers(' '.join(companies))  # .Tickers() function is used to fetch the data for all of the companies in the list within the industry for that iteration of the for loop
    # creating lists to store the  market capitalisations, P/E ratios and dividend yields per company
    market_caps = []
    pe_ratios = []
    dividend_yields = []

    # getting the data for each company within the industry specified for the parent for loop
    for company in companies:
        info = data.tickers[company].info  # retrieveing detailed information about the company

        # Adding the market cap, P/E ratio and dividend yield for the current company to the lists defined above
        # getting the market cap and trailing P/E ratio, defaulting to 0 if data is missing
        market_caps.append(info.get('marketCap', 0))  
        pe_ratios.append(info.get('trailingPE', 0))
        #calculating the dividend yield as a percentage
        dividend_yields.append(info.get('dividendYield', 0) * 100 if info.get('dividendYield') else 0)

    # calculating the averages for the market cap, P/E ratio and dividend yield for the industry using the values added to the 3 lists for each company
    # first the 3 values are added together then the length of list is used to divide the sum giving the average for each industry
    avg_market_cap = sum(market_caps) / len(market_caps)
    avg_pe_ratio = sum(pe_ratios) / len(pe_ratios)
    avg_dividend_yield = sum(dividend_yields) / len(dividend_yields)

    # adding the results for the industry to the summary list
    industry_summary.append({
        'Industry': industry,  # industry name
        'Avg Market Cap ($B)': round(avg_market_cap / 1e9, 2),  # average market cap in billions
        'Avg P/E Ratio': round(avg_pe_ratio, 2),  # average price-to-earnings ratio
        'Avg Dividend Yield (%)': round(avg_dividend_yield, 2)  # Average dividend yield percentage
    })

# creating a DataFrame to store and display results 
summary_df = pd.DataFrame(industry_summary)  # converting summary list into a DataFrame
print(summary_df)  # displaying the summary table
